In [1]:
import torch.nn as nn
import numpy as np
import gym
from seagul.rl.ars.meta_ars import MetaARSAgent
import matplotlib.pyplot as plt
import torch
import dill
import seagul.envs
import pybullet_envs
import pickle
from scipy.signal import find_peaks

from common import *

%load_ext autoreload
%autoreload 2

from IPython.display import Audio
# sound_file = '/home/sgillen/Downloads/chime.wav'
# Audio(filename=sound_file, autoplay=True)

/home/sgillen/work/seagul/seagul/envs/__init__.py:58: UserWarning: Warning, pybullet envs not installed
  warnings.warn("Warning, pybullet envs not installed")


In [2]:
meta_agent = pickle.load(open("agents/meta_hmap_hopper_flat4", "rb"))
agent0 = meta_agent.agents[4]

In [4]:
env_name = "hurdle_hopper-v0"
seed = 4
env = gym.make(env_name,gap_length=50)
#env = gym.make("Hopper-v2")

#agent = meta_agent.agents[seed]
obs, act, rew, _ = do_rollout(env, agent0.model, render=True, ep_length=1000)
print(sum(rew))
plt.plot(obs);

Creating window glfw


KeyboardInterrupt: 

In [ ]:
# agent0.env_config={"slope_set":[0], 'random':False}
# agent0.postprocessor = mdim_div
# agent0.learn(100);
# Audio(filename=sound_file, autoplay=True)

In [ ]:
# import copy
# agent_list = []
# slope_set = [-.01, 0.0, .01]
# agent = copy.deepcopy(agent0)
# #agent.postprocessor = mdim_div
# agent.env_config = {"slope_set":[slope, 0.0]}
# agent.learn(200)
# agent_list.append(agent)
# Audio(filename=sound_file, autoplay=True)

In [3]:
import copy
#agent_list = []
slope_set = [0.0, 0.1]
for slope in slope_set:
    agent = copy.deepcopy(agent0)
    #agent.postprocessor = mdim_div
    agent.env_config = {"slope_set":[slope], "random":False}
    agent.learn(200)
    agent_list.append(agent)
    
Audio(filename=sound_file, autoplay=True)

ERROR! Session/line number was not unique in database. History logging moved to new session 7008
0 : mean return: 863.9389700356068, fps:31597.386157512556
10 : mean return: 992.5042509865459, fps:33164.43966114157
20 : mean return: 2005.1598302272257, fps:33449.02353024659
30 : mean return: 1106.0630492604896, fps:34447.29216981553
40 : mean return: 2107.8959296687426, fps:35283.99194213001
50 : mean return: 1743.4295077365039, fps:35093.58921787952
60 : mean return: 1920.4656765734253, fps:33661.78790180956
70 : mean return: 2087.0606928484167, fps:32495.05655217124
80 : mean return: 1961.0550341728424, fps:34863.65650454531
90 : mean return: 1794.8185458236117, fps:32125.086490496204
100 : mean return: 1564.0814903204307, fps:36640.8467310806
110 : mean return: 2197.9717420399256, fps:36874.95975189189
120 : mean return: 2007.0274750144993, fps:35955.857194233955
130 : mean return: 1799.631725454206, fps:35976.957645397975
140 : mean return: 2234.987972105448, fps:35722.89515553069


NameError: name 'agent_list' is not defined

In [ ]:
import xarray as xr
slope_set = [0.0, 0.1, 0.02]
n_runs = 5
data = xr.DataArray(np.zeros((len(slope_set), len(agent_list), n_runs)),
                    dims=("slope", "agent", "run"))

for i, slope in enumerate(slope_set):
    env.unwrapped.slope_set = [slope] 
    for j, agent in enumerate(agent_list):
        for run in range(n_runs):
            obs,act,rew,_ = do_rollout(env, agent.model, render=False, return_on_done=True)
            data.loc[i, j, run] = sum(rew).item()

In [ ]:
data.mean(dim='run')

In [ ]:
env.unwrapped.slope_set = [-.02]
obs,act,rew,_ = do_rollout(env, agent_list[0].model, render=True, return_on_done=True)

In [ ]:
print(len(rew))
print(sum(rew))

In [ ]:
#normal_seed  = 6
#noise_seed = 0 or 8

def my_reset(env, point):
    env.reset()
    reset_state = np.copy(point)
    reset_state[0] += (env.get_height(5) - env.model.hfield_size[0,2]/2)
    qpos = np.concatenate((np.array([5.0]), reset_state[:5]))
    qvel = reset_state[5:-1]
    env.unwrapped.set_state(qpos, qvel)
    return env.unwrapped._get_obs()

def fail_cond(env):
    
    s = env.unwrapped.state_vector()
    posafter, height, ang = env.unwrapped.sim.data.qpos[0:3]
    done = not (np.isfinite(s).all() and (np.abs(s[2:]) < 100).all() and
                    (height > .7) and (abs(ang) < .4))
    
    # set done = true if anything but the foot and ground are in contact.
    done = False
    ncon = env.unwrapped.sim.data.ncon
    for i in range(ncon):
        geom1 = env.unwrapped.sim.data.contact[i].geom1
        geom2 = env.unwrapped.sim.data.contact[i].geom2
        if not (geom1 == 4 or geom1 == 0):
            done = True
        if not (geom2 == 4 or geom2 == 0):
            done = True
            
    return done

#     #tilted = np.abs(ob[:,1]) > .2 
#     short = ob[:,0] < .7
#     #return np.logical_or(tilted, short)
#     return short

from common import *
from collections import namedtuple

class MeshBox:
    def __init__(self, state, iden, freq, npols):
        self.state = state
        self.id = iden
        self.freq = freq
        
        self.succ_list = [[] for _ in range(npols)]
        self.pred_list = []
        self.succ_rew_list =  [[] for _ in range(npols)]
            
        self.ttl = 0
        self.opt_controller = None
        


In [ ]:
env = gym.make("hmap_hopper-v0", slope_set=[0])

delta =  1;
#pols = #policy_dict[post_names[1]][0:3]
pols = [agent.model for agent in agent_list]
npol = copy.deepcopy(pols[1])
npol.state_std[-1] = .01

mesh = BoxMesh(delta); mdim_l = []


fail_state = np.ones_like(env.reset())*100
fail_id = 0
mesh[fail_state] = MeshBox(fail_state, iden = 0, freq = 1, npols=len(pols))

for policy, slope in zip(pols, [-.01, 0, .01]):
    env.unwrapped.slope_set = [slope]
    #env.reset()
    obs, acts, rews, uc_obs = do_rollout(env, policy, ep_length=1000, return_on_done=True, render=False)
    peak_idx,_ = find_peaks(uc_obs[:,0], distance=20)


    for o in obs[peak_idx[2:],:]:
        key = (o - npol.state_mean)/npol.state_std

        if key in mesh:
            mesh[key].freq +=1
        else:
            mesh[key] =  MeshBox(o, iden = len(mesh), freq = 1, npols=len(pols))


    plt.plot(obs[:,0]);
    peak_idx,_ = find_peaks(obs[:,0], distance=20)

    for i in peak_idx[2:]:
        plt.axvline(x=i, color='r')

    print(fail_cond(env))

    print(f"{len(peak_idx[2:])} poincare snapshots, {len(mesh)} states in mesh")
    plt.figure();

In [ ]:
from multiprocessing import Pool
from itertools import repeat

slope_mesh = BoxMesh(.01)
slope_mesh[[.01]] = 1
slope_mesh[[0]] = 1
slope_mesh[[-.01]] = 1

In [ ]:
def do_explore_rollout(args):
    policy, init_state, trial_num = args
    slope = init_state[-1]
    force = np.zeros(2)
    env.seed(trial_num)
    env.unwrapped.slope_set = [slope]
    np.random.seed(trial_num)
    env.seed(trial_num)
    obs, acts, rews, x = do_push_rollout_from_state(env, policy, init_state, force, render=False)    
    
    return np.asarray(obs), np.asarray(rews)

def do_push_rollout_from_state(env, policy, init_state, force, num_steps=200, force_steps = 0, render=False):
    act_list = []
    obs_list = []
    rew_list = []
    x_list = []
    obs = my_reset(env, init_state)

    done = False
    obs_size = env.observation_space.shape[0]
    act_size = env.action_space.shape[0]

    for cur_step in range(num_steps):
        if render:
            env.render()
            time.sleep(.01)
        
        obs_list.append(np.copy(obs))

        x_list.append(env.unwrapped.sim.data.qpos[0])
        
        if cur_step < force_steps:
            env.unwrapped.sim.data.xfrc_applied[1,:] = np.array([force[0],0,force[1],0,0,0])
        else:
            env.unwrapped.sim.data.xfrc_applied[1,:] = np.zeros(6)
        
        act,_,_,_ = policy.step(obs)
        #act += torch.randn(act_size)*act_std
        obs, rew, done, _ = env.step(act)
        
        act_list.append(np.copy(act))
        rew_list.append(rew)
        
        if done:
            break
        
    ep_obs = np.stack(obs_list)
    ep_act = np.stack(act_list)
    ep_rew = np.array(rew_list)
    ep_rew = ep_rew.reshape(-1, 1)

    return ep_obs, ep_act, ep_rew, None


start = time.time()
keys = list(mesh.keys())[1:]
i = 0 
slope_set = [-.01, 0, .01]
with Pool(12, maxtasksperchild=1000) as pool:
    for init_key in keys:    
        i+=1 
        
        init_state = mesh[init_key].state
        init_id = mesh[init_key].id
        
        state_iter = repeat(init_state, len(pols))
        rollout_seed_iter = range(i*len(pols), i*len(pols)+len(pols))

        for j, res in enumerate(pool.imap(do_explore_rollout, zip(pols, state_iter, rollout_seed_iter))):
            obs, rews = res
    
            peak_idx,_ = find_peaks(obs[:,0], distance=20)
            

            if fail_cond(env) or not peak_idx.any():
                new_id = fail_id
                new_key = fail_state
                mesh[init_key].succ_list[j].append(new_id)
                mesh[init_key].succ_rew_list[j].append(0)
                mesh[new_key].pred_list.append(init_id)
                        
            elif(peak_idx.any()):
                peak_obs = obs[peak_idx[0],:]
                for slope in slope_set: 
                    peak_obs[-1] = slope
                    new_key = ((peak_obs - np.array(npol.state_mean))/np.array(npol.state_std)).squeeze()
                    if new_key in mesh:
                        new_id = mesh[new_key].id
                        mesh[new_key].freq += 1
                    else:
                        mesh[new_key] = MeshBox(np.copy(peak_obs), len(mesh), 1, npols=len(pols))
                        new_id = len(mesh)-1
                        keys.append(mesh.__keytransform__(new_key))
                        
                    mesh[init_key].succ_list[j].append(new_id)
                    mesh[init_key].succ_rew_list[j].append(sum(rews))
                    mesh[new_key].pred_list.append(init_id)
            else:
                print("how???")
           
                
        if i % 1000 == 0:
            print(f"i; {i}, mdim: {len(keys)}, ratio: {i/len(keys)}")
            #print(f"i; {i}, iden: {len(iden_mesh)}, mdim: {len(mdim_mesh)}, ratio: {len(mdim_mesh)/len(iden_mesh)}")
        
print(time.time() - start)

In [ ]:
print(mesh)

key = list(mesh.keys())[1]

mesh_map = {state.id:state for state in mesh.values()}
npols=3
heights = np.array([m.state[-1] for m in mesh_map.values()])
plt.hist(heights[1:],bins=3)

In [ ]:
# Recursive TTL

def V(mesh_id):
    mesh_state = mesh_map[mesh_id]
    
    if mesh_state.visited:
        #print(f"{mesh_id} limit cycle detected")
        mesh_state.ttl = float('inf')
        return float('inf')
        
    if (np.asarray(mesh_state.state) == fail_state).all():
        #print(f"{mesh_id} returning 0")
        return 0
    
    elif mesh_state.ttl == 0:
        #print(f"{mesh_id} descending into {mesh_state.succ_list}")
        mesh_state.visited = True
        succ_V_list = [[] for _ in range(npols)]
        inf_list = []
        
        for next_controller_id, controller_succ_list in enumerate(mesh_state.succ_list):
            for next_state_id in controller_succ_list:
                succ_V_list[next_controller_id].append(V(next_state_id))
                
#         for controller_id, rew_list in next_list:
#             limit_idx = [i for i,ttl in enumerate(rew_list) if ttl == float('inf')]
#             if len(limit_idx):
#                 mesh_state.ttl = float('inf')
#                 limit_rewards = [mesh_state.succ_rew_list[i] for i in limit_idx]
#                 mesh_state.opt_controller = mesh_state.succ_rew_list.index(max(limit_rewards))

        else:
            min_slope_V = [min(thing) for thing in succ_V_list]
            max_controller_V = max(min_slope_V)
            curr_r  = max(max(mesh_state.succ_rew_list))
            
            mesh_state.ttl =  curr_r + max_controller_V
            mesh_state.opt_controller = min_slope_V.index(max_controller_V)

        #print(f"{mesh_id} resulting list was {next_list} , new ttl is {mesh_state.ttl}")
        return mesh_state.ttl
        
    else:
        #print(f"{mesh_id} returning ttl of {mesh_state.ttl}")
        return mesh_state.ttl
    
    
# Iterative TTL   
        
for state in mesh.values():
    state.ttl = 0
    state.visited = False
    
    
V(10)

In [ ]:
def do_opt_rollout_from_state(env, pols, init_state, obs_std=0, act_std=0, num_steps=1000, render=True):
    act_list = []
    obs_list = []
    rew_list = []
    val_list = []

    obs = my_reset(env, init_state)
    uc_obs = np.concatenate([env.sim.data.qpos.flat[1:], env.sim.data.qvel.flat])
    done = False
    obs_size = env.observation_space.shape[0]
    act_size = env.action_space.shape[0]
    pi = 1
    policy = pols[pi]


    
    out = 0
    inm = 0
    while not done:
        if render:
            env.render()
            time.sleep(.01)
            
        obs_key = (obs - npol.state_mean)/npol.state_std
        if not obs_key in mesh:
            out+=1
        else:
            if mesh[obs_key].opt_controller is None:
                V(mesh[obs_key].id)
            
            inm +=1
            print(mesh[obs_key].opt_controller)
            policy = pols[mesh[obs_key].opt_controller] 
            print(f"going from pol {pi} to {mesh[obs_key].opt_controller}" )
            pi = mesh[obs_key].opt_controller
            

        #policy = pols[0]

        obs += np.random.randn(obs_size)*obs_std
        obs_list.append(np.copy(obs))
        
       
        try:
            val_list.append(V(mesh[obs_key].id))
        except:
            pass
        
        act,_,_,_ = policy.step(obs)
        act += np.random.randn(act_size)*act_std
        obs, rew, done, _ = env.step(act)
        
        act_list.append(np.copy(act))
        rew_list.append(rew)
        
    ep_obs = np.stack(obs_list)
    ep_act = np.stack(act_list)
    ep_rew = np.array(rew_list).reshape(-1,1)
    ep_vals = np.array(val_list).reshape(-1,1)

    
    print(out, inm)
    torch.autograd.set_grad_enabled(True)
    return ep_obs, ep_act, ep_rew, ep_vals

#mesh_map = {state.id:state for state in mesh.values()}

env.unwrapped.slope_set = [0.0]#[0.00, 0.01, -0.01]
env.unwrapped.random=True
obs,act,rews,vals = do_opt_rollout_from_state(env, pols, mesh_map[10].state, render=True)

In [ ]:
num_trials = 250000
X = [(point.state - npol.state_mean)/npol.state_std for point in mesh_map.values()]
X = np.array(X, dtype=np.float32)

Yl = []
for x in X:
    if x[-1] < 0:
        Yl.append(0)
    elif x[-1] == 0:
        Yl.append(1)
    elif  x[-1] >  0:
        Yl.append(2)
    else:
        panic
        
    

Y = np.stack(Yl)
Y = np.array(Y,dtype=np.long)

In [ ]:
from seagul.nn import fit_model
net = MLP(12, 3, 2, 32)
loss_hist = fit_model(net, X, Y, 100, batch_size=2048, loss_fn=torch.nn.CrossEntropyLoss())

plt.plot(loss_hist)

In [ ]:
print(mesh_map[5].state)
net(np.array((mesh_map[1].state - npol.state_mean)/npol.state_std, dtype=np.float32))

In [ ]:
mesh_map[1].state

In [ ]:
def do_nn_rollout_from_state(env, pols, init_state, obs_std=0, act_std=0, num_steps=1000, render=True):
    act_list = []
    obs_list = []
    rew_list = []
    val_list = []

    obs = my_reset(env, init_state)
    uc_obs = np.concatenate([env.sim.data.qpos.flat[1:], env.sim.data.qvel.flat])
    done = False
    obs_size = env.observation_space.shape[0]
    act_size = env.action_space.shape[0]
    pi = 1; old_pi = pi
    policy = pols[pi]

    while not done:
        if render:
            env.render()
            time.sleep(.01)
    
        preds = net(np.array(obs,dtype=np.float32))
        pi = torch.argmax(preds)
#         if pi != old_pi:
#             print(f"went from {old_pi} to {pi}")

        #print(env.get_height(0), env.get_height(1), env.get_height(2), obs[-1])
        old_pi = pi
        policy = pols[pi]

        #policy = pols[0]

        obs += np.random.randn(obs_size)*obs_std
        obs_list.append(np.copy(obs))
        
       
        try:
            val_list.append(V(mesh[obs_key].id))
        except:
            pass
        
        act,_,_,_ = policy.step(obs)
        act += np.random.randn(act_size)*act_std
        obs, rew, done, _ = env.step(act)
        
        act_list.append(np.copy(act))
        rew_list.append(rew)
        
    ep_obs = np.stack(obs_list)
    ep_act = np.stack(act_list)
    ep_rew = np.array(rew_list).reshape(-1,1)
    ep_vals = np.array(val_list).reshape(-1,1)

    
    torch.autograd.set_grad_enabled(True)
    return ep_obs, ep_act, ep_rew, ep_vals

#mesh_map = {state.id:state for state in mesh.values()}

env.unwrapped.slope_set = [0.01]
env.unwrapped.random=False
obs,act,rews,vals = do_nn_rollout_from_state(env, pols, mesh_map[10].state, render=True)

In [ ]:
plt.plot(obs[:,-1])

In [ ]:
torch.argmax??